# Malaysia COVID-19 Cases and Vaccination

Data taken from [**Malaysia's Ministry of Health**](https://github.com/MoH-Malaysia/covid19-public) and [**COVID-19 Immunisation Task Force (CITF)**](https://github.com/CITF-Malaysia/citf-public) as of **`6-10-2021`** (cut-off date).

1. [covid19-public](https://github.com/MoH-Malaysia/covid19-public/tree/9a8482527c486effa37900756631f843af4d6373)
2. [citf-public](https://github.com/CITF-Malaysia/citf-public/tree/46f91d6e18e38a24034bf8f791cd279b31f94f24)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import math
import pickle

from imblearn.over_sampling import SMOTE
from collections import Counter

## 1. Loading Cleaned Datasets

Loading cleaned datasets that was pickled in `EDA_Epidemic.ipynb`, `EDA_Vaccination_&_Registration.ipynb`, and `EDA_MySejahtera.ipynb`

In [4]:
##### EPIDEMIC dataset #####
## Cases and Testing
cases_malaysia = pickle.load(open('pickle_files/cases_malaysia.pkl', 'rb'))
cases_malaysia_cluster = pickle.load(open('pickle_files/cases_malaysia_cluster.pkl', 'rb'))
cases_state = pickle.load(open('pickle_files/cases_state.pkl', 'rb')) 

tests_malaysia = pickle.load(open('pickle_files/tests_malaysia.pkl', 'rb')) 
tests_state = pickle.load(open('pickle_files/tests_state.pkl', 'rb')) 

## Deaths
deaths_malaysia = pickle.load(open('pickle_files/deaths_malaysia.pkl', 'rb'))
deaths_state = pickle.load(open('pickle_files/deaths_state.pkl', 'rb')) 

## Static Data
population = pd.read_csv('dataset/static/population_moh.csv')

##### VACCINATION & REGISTRATION dataset #####
## Adverse Events Following Immunization (AEFI)
aefi = pickle.load(open('pickle_files/aefi.pkl', 'rb'))
aefi_serious = pickle.load(open('pickle_files/aefi.pkl', 'rb'))

## Vaccination
vax_malaysia = pickle.load(open('pickle_files/vax_malaysia.pkl', 'rb'))
vax_state = pickle.load(open('pickle_files/vax_state.pkl', 'rb'))

## Registration
vaxreg_malaysia = pickle.load(open('pickle_files/vaxreg_malaysia.pkl', 'rb'))
vaxreg_state = pickle.load(open('pickle_files/vaxreg_state.pkl', 'rb'))

##### MYSEJAHTERA dataset #####
checkin_malaysia = pickle.load(open('pickle_files/checkin_malaysia.pkl', 'rb'))
checkin_malaysia_time = pickle.load(open('pickle_files/checkin_malaysia_time.pkl', 'rb'))
checkin_state = pickle.load(open('pickle_files/checkin_state.pkl', 'rb'))
trace_malaysia = pickle.load(open('pickle_files/trace_malaysia.pkl', 'rb'))

In [11]:
aefi.tail(3)

,date,vaxtype,daily_total,daily_nonserious_mysj,daily_nonserious_npra,daily_serious_npra,daily_nonserious_mysj_dose1,daily_nonserious_mysj_dose2,d1_site_pain,d1_site_swelling,...,d2_site_redness,d2_tiredness,d2_headache,d2_muscle_pain,d2_joint_pain,d2_weakness,d2_fever,d2_vomiting,d2_chills,d2_rash
579,2021-09-29,pfizer,2240,2151,81,8,2007,144,1553,417,...,11,63,72,44,29,42,32,22,21,5
580,2021-09-29,cansino,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581,2021-09-29,sinovac,158,144,11,3,131,13,61,8,...,1,5,2,4,2,6,2,1,0,0


In [6]:
from tslearn.utils import to_time_series_dataset
cluster = 3
df = deaths_pT_new.iloc[:,35:]
X_train = df.rolling(7, axis=1, min_periods=1).mean().fillna(0)
colors = ['blue', 'red', 'green']
names = ['blue cluster','red cluster','green cluster']
seed = 1
np.random.seed(seed)
X_train = to_time_series_dataset(X_train.copy())

print('COVID-19 deaths vs time curves')
km = TimeSeriesKMeans(n_clusters=cluster, verbose=True, random_state=seed,
                         max_iter=10)

y_pred = km.fit_predict(X_train)
clusters = pd.Series(data=y_pred, index=df.index)

f, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, sharey=True,figsize=(5, 8))

for yi,cl,xs in zip(range(cluster),[2,1,0],[ax1,ax2,ax3]):
    data = df.rolling(7, axis=1, min_periods=1).mean().fillna(0).loc[clusters[clusters == cl].index]
    data.T.plot(legend=False, alpha=.2,color='black', ax=xs)
    data.mean(axis=0).plot(linewidth=3., color=colors[cl], ax=xs)
    n = len(data)
    print('{}, N = {}'.format(names[cl], n))

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax3.spines['right'].set_visible(False)

f.subplots_adjust(hspace=0)
plt.ylim(-0.02, 3.5)
plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)


,date,state,deaths_new,deaths_bid,deaths_new_dod,deaths_bid_dod,deaths_pvax,deaths_fvax,deaths_tat
0,2020-03-17,Johor,1,0,1,0,0,0,0
1,2020-03-17,Kedah,0,0,0,0,0,0,0
2,2020-03-17,Kelantan,0,0,0,0,0,0,0
3,2020-03-17,Melaka,0,0,0,0,0,0,0
4,2020-03-17,Negeri Sembilan,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
9083,2021-10-05,Selangor,14,2,0,0,0,0,4
9084,2021-10-05,Terengganu,5,0,1,0,0,0,3
9085,2021-10-05,W.P. Kuala Lumpur,4,1,0,0,0,0,2
9086,2021-10-05,W.P. Labuan,0,0,0,0,0,0,0
